# Extração de dados de normas

Neste notebook são descritos os passos de preparação dos dados normas a serem utilizados pelos modelos de classificação textual.

### Bibliotecas úteis

In [10]:
import pandas as pd
import pickle
from unidecode import unidecode
import numpy as np

%reload_ext autoreload
%autoreload 1

%aimport dataset

Using TensorFlow backend.


### Carregando dataset de normas

In [2]:
normas = pd.read_csv('data/normas_raw.tsv', sep='\t', encoding='cp1252', 
                     error_bad_lines=False).replace(['(null)'], value=[None])

# Alguns textos aparecem com o valor '(null)'. Precisamos excluir esses casos.
normas = normas[normas.TEXTO.notnull()]
print(normas.shape)
normas.head()

b'Skipping line 5983: expected 18 fields, saw 19\n'


(257178, 18)


,IDENORMA,IDETIPONORMA,IDESITUACAONORMA,TEXNUMERONORMA,ANONORMA,NUMSESSAOLEGISLATIVA,DATASSINATURA,TEXAPELIDO,TEXCOLECAO,TEXORIGENS,TEXSIGLASORIGENS,TEXEPIGRAFE,TEXEPIGRAFEREFERENCIA,NOMTIPONORMA,SIGTIPONORMA,TEXTO,INDEXACAO,IDETEMA
0,515858,33823,None,20929,1932,None,1932-01-11 00:00:00,None,Legislação Superior,Poder Executivo,PE,"Decreto nº 20.929, de 11 de Janeiro de 1932",Decreto nº 20929 de 11 de Janeiro de 1932,Decreto,decret,<HTML><BODY> <CHR13 /> <CHR10 /> <P>O Chefe d...,None,None
1,407288,33823,None,65957,1969,None,1969-12-23 00:00:00,None,Legislação Superior,Poder Executivo,PE,"Decreto nº 65.957, de 23 de Dezembro de 1969",Decreto nº 65957 de 23 de Dezembro de 1969,Decreto,decret,<CHR13 /> <CHR10 /> <CHR13 /> <CHR10 /> <...,CRÉDITO SUPLEMENTAR - Aberturas <CHR13 /> <CH...,None
2,426085,33823,None,77667,1976,None,1976-05-24 00:00:00,None,Legislação Superior,Poder Executivo,PE,"Decreto nº 77.667, de 24 de Maio de 1976",Decreto nº 77667 de 24 de Maio de 1976,Decreto,decret,<HTML><BODY> <CHR13 /> <CHR10 /> <P>O PRESIDE...,CONFISCO DE BENS - Atos pessoais <CHR13 /> <C...,None
3,501503,33823,None,15867,1922,None,1922-11-29 00:00:00,None,Legislação Superior,Poder Executivo,PE,"Decreto nº 15.867, de 29 de Novembro de 1922",Decreto nº 15867 de 29 de Novembro de 1922,Decreto,decret,<HTML><BODY> <CHR13 /> <CHR10 /> <P>O Preside...,None,None
4,498505,33823,None,3582,1900,None,1900-02-03 00:00:00,None,Legislação Superior,Poder Executivo,PE,"Decreto nº 3.582, de 3 de Fevereiro de 1900",Decreto nº 3582 de 3 de Fevereiro de 1900,Decreto,decret,<HTML><BODY> <CHR13 /> <CHR10 /> <P>O Preside...,None,None


### Removendo tags HTML

In [5]:
%%time
normas['INDEXACAO'] = normas['INDEXACAO'].map(lambda x: x.replace('<CHR13 />', '|') if x else None)
normas['INDEXACAO'] = normas['INDEXACAO'].map(lambda x: dataset.preprocess_html(x) if x else None)
normas['TEXTO'] = normas['TEXTO'].map(lambda x: dataset.preprocess_html(x))

Wall time: 13min 11s


In [6]:
print(normas.shape[0], 'normas')
normas[['IDENORMA', 'TEXTO', 'INDEXACAO']].head()

257178 normas


,IDENORMA,TEXTO,INDEXACAO
0,515858,O Chefe do Governo Provisorio da Republica...,None
1,407288,"O PRESIDENTE DA REPUBLICA, usando ...",CREDITO SUPLEMENTAR - Aberturas | MINISTERIO...
2,426085,"O PRESIDENTE DA REPUBLICA , no uso das atr...",CONFISCO DE BENS - Atos pessoais | ENRIQUECI...
3,501503,O Presidente da Republica dos Estados Unid...,None
4,498505,O Presidente da Republica dos Estados Unid...,None


### Carregando nomes de temas

In [7]:
temas = pd.read_csv('data/temas.csv', sep=';')
temas['CLASS'] = temas.index
print(temas.shape)
temas.head()

(42, 3)


,IDETEMA,TEMA,CLASS
0,34,Administração Pública,0
1,64,"Agricultura, Pecuária e Pesca",1
2,35,Arte e Cultura,2
3,62,Ciência e Tecnologia,3
4,36,"Ciência, Tecnologia e Informática",4


### Adicionando nomes de temas à tabela de normas

In [8]:
# convertendo tipo de IDETEMA para numerico
normas['IDETEMA'] = pd.to_numeric(normas.IDETEMA)

# merge das tabelas
normas_temas = normas.merge(temas, on='IDETEMA', how='left')

print(normas_temas.shape)
assert normas_temas.shape[0] == normas.shape[0]
normas_temas[normas_temas.TEMA.notnull()].head()

(257178, 20)


,IDENORMA,IDETIPONORMA,IDESITUACAONORMA,TEXNUMERONORMA,ANONORMA,NUMSESSAOLEGISLATIVA,DATASSINATURA,TEXAPELIDO,TEXCOLECAO,TEXORIGENS,TEXSIGLASORIGENS,TEXEPIGRAFE,TEXEPIGRAFEREFERENCIA,NOMTIPONORMA,SIGTIPONORMA,TEXTO,INDEXACAO,IDETEMA,TEMA,CLASS
12,609841,33828,None,54560,2010,None,2010-12-27 00:00:00,None,Legislação Superior,Poder Executivo,PE,Decreto de 27 de Dezembro de 2010,Decreto de 27 de Dezembro de 2010,Decreto Sem Número,decret_sn,"O PRESIDENTE DA REPUBLICA, no uso ...",REFORMA AGRARIA - Imovel rural - Desapropriaca...,51.0,Política Fundiária,30.0
45,605183,33823,None,7149,2010,None,2010-04-08 00:00:00,None,Legislação Superior,Poder Executivo,PE,"Decreto nº 7.149, de 8 de Abril de 2010",Decreto nº 7149 de 8 de Abril de 2010,Decreto,decret,"O PRESIDENTE DA REPUBLICA,...",ATOS INTERNACIONAIS - Republica Democratica do...,55.0,Relações Internacionais,34.0
54,599446,33823,None,7059,2009,None,2009-12-29 00:00:00,None,Legislação Superior,Poder Executivo,PE,"Decreto nº 7.059, de 29 de Dezembro de 2009",Decreto nº 7059 de 29 de Dezembro de 2009,Decreto,decret,O PRESIDENTE DA RE...,SEGURO RURAL - Subvencao economica - Premio - ...,51.0,Política Fundiária,30.0
74,609851,33828,None,54565,2010,None,2010-12-27 00:00:00,None,Legislação Superior,Poder Executivo,PE,Decreto de 27 de Dezembro de 2010,Decreto de 27 de Dezembro de 2010,Decreto Sem Número,decret_sn,"O PRESIDENTE DA REPUBLICA,...",REFORMA AGRARIA - Imovel rural - Desapropriaca...,51.0,Política Fundiária,30.0
85,775237,33823,None,7906,2013,None,2013-02-04 00:00:00,None,Legislação Superior,Poder Executivo,PE,"Decreto nº 7.906, de 4 de Fevereiro de 2013",Decreto nº 7906 de 4 de Fevereiro de 2013,Decreto,decret,"A PRESIDENTA DA REPUBLICA, no uso da atri...",ATOS INTERNACIONAIS - Reino dos Paises Baixos ...,43.0,Direito Penal e Processual Penal,16.0


### Salvando normas com temas e indexação 

In [9]:
normas_temas.to_csv('data/normas.csv', index=False, encoding='utf-8')